## Lesson 3.3 Practice: Preparing CAS Tables

### Documentation:
- [SAS Viya Actions and Action Sets by Name and Product](https://go.documentation.sas.com/doc/en/pgmsascdc/v_018/allprodsactions/titlepage.htm)
- [SAS Scripting Wrapper for Analytics Transfer (SWAT)](https://sassoftware.github.io/python-swat/index.html)

1. Import packages and connect to the CAS server.

In [21]:
## Import packages
import swat
import pandas as pd

## Set options
pd.set_option('display.max_columns', None)

## Connect to CAS
conn = swat.CAS('server.demo.sas.com', 30571, 'student', 'Metadata0', name = 'py03p02')

2. Confirm that the **LOANS_RAW** CAS table is available in memory. If not, load the **loans_raw.sashdat** file into the **Casuser** caslib.

#### Click the three dots below to reveal the solution.

(solution 1) Manually check to see if the table is in-memory.

In [24]:
## View available in-memory tables
ti = conn.tableInfo(caslib = 'casuser')
display(ti)

## Create a function to load the loans_raw.sashdat file into memory if necessary
def loadLoans():
    lt = conn.loadTable(path = 'loans_raw.sashdat', caslib = 'PIVY',
                        casOut = {'replace' : True})
    return lt

## Execute the loadLoans function if necessary
loadLoans()

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,LOANS_RAW,16311047,20,0,utf-8,2022-03-21T12:37:44+00:00,2022-03-21T12:37:44+00:00,2022-03-21T12:37:44+00:00,UTF8,1.963485e+09,1.963485e+09,1.963485e+09,0,0,0,0,loans_raw.sashdat,PIVY,0,student,,2021-12-17T18:40:14+00:00,1.955386e+09


NOTE: Cloud Analytic Services made the file loans_raw.sashdat available as table LOANS_RAW in caslib CASUSER(student).


[caslib]

 'CASUSER(student)'

[tableName]

 'LOANS_RAW'

[casTable]

 CASTable('LOANS_RAW', caslib='CASUSER(student)')

+ Elapsed: 0.0871s, user: 0.0288s, sys: 0.058s, mem: 4.98mb

(solution 2, advanced) Use the [table.tableExists](https://go.documentation.sas.com/doc/en/pgmsascdc/v_018/caspg/cas-table-tableexists.htm) action to check to see if a table is loaded in-memory.  

In [25]:
## Use the tableExists action to check to see if a table is loaded in-memory
if conn.tableExists(name = 'loans_raw', caslib = 'casuser')['exists'] == 0:
    conn.loadTable(path = 'loans_raw.sashdat', caslib = 'PIVY',
                   casOut = {'replace' : True})
else:
    print('Table is already loaded in-memory')

Table is already loaded in-memory


3. Reference the **LOANS_RAW** CAS table in the variable **tbl**. Then preview five rows.

In [26]:
tbl = conn.CASTable('loans_raw', caslib = 'casuser')
tbl.head()

,ID,AccNumber,Year,Month,Day,CurrentDate,SalaryGroup,Age,Salary,EmpLength,Category,Amount,InterestRate,LoanLength,LoanGrade,LoanStatus,LastPurchase,Cancelled,CancelledReason,Promotion
0,I-00006307127350009978,CL3246299535967411-4949000,2022.0,7.0,13.0,31DEC2022,"Less than $50,000",30.0,9128.0,11.0,Car Loan,14173.66,4.26,5.0,C,Current,NaT,0.0,,0.0
1,I-00006307127350009978,PKEO-1058-5781-4683-1737,2020.0,7.0,16.0,31DEC2022,"Less than $50,000",30.0,9128.0,11.0,Credit Card,2883.72,16.78,999999.0,C,Late,2022-12-16,0.0,,0.0
2,I-00006307127350009978,M9708726115059108-92217000,2016.0,3.0,15.0,31DEC2022,"Less than $50,000",30.0,9128.0,11.0,Mortgage,13642.37,5.00,30.0,C,Current,NaT,0.0,,0.0
3,G-00005046456530690193,PKEO-0775-3531-9166-5132,2020.0,2.0,10.0,31DEC2022,"$50,001 - $100,000",46.0,92000.0,10.0,Credit Card,10957.10,13.62,999999.0,B,Current,2022-03-19,0.0,,0.0
4,C-00006496445944067091,PKEO-0832-4079-2767-3394,2013.0,5.0,5.0,31DEC2022,"Less than $50,000",28.0,17029.0,14.0,Credit Card,5257.41,21.83,999999.0,B,Current,2014-08-07,0.0,,0.0


4. Create a new CAS table with the following requirements:
- The new table should contains rows where **Category** equals *Car Loan* and **LoanStatus** is not equal to *Fully Paid*. (For a not equal to operator, use **ne**).
- Create a new column named **AccOpenDate** with the loan open date using the **Month**, **Day**, and **Year** columns and the SAS MDY function.
- Create a new column named **LoanLengthMonths** by multiplying **LoanLength** by 12.
- The new table should have the following columns: **ID**, **AccNumber**, **Amount**, **AccOpenDate**, **LoanLengthMonths**, **LoanStatus**, and **Category**.

(solution 1) Preparing the data using **CASTable** object parameters.

In [27]:
## Subset the data by adding the where parameter to the CASTable object
tbl.where = 'Category = "Car Loan" and LoanStatus ne "Fully Paid"'

## Create the calculated columns by adding the computedVarsProgram parameter to the CASTable object
calcAccOpenDate = 'AccOpenDate = mdy(Month, Day, Year);'
calcLoanLengthMonths = 'LoanLengthMonths = LoanLength * 12;'
tbl.computedVarsProgram = calcAccOpenDate + calcLoanLengthMonths

## Specify the columns to include by adding the vars parameter to the CASTable object
tbl.vars = ['ID', 'AccNumber', 'Amount', 'AccOpenDate', 'LoanLengthMonths','LoanStatus', 'Category']


## Preview the CAS table with the parameters
tbl.head()

,ID,AccNumber,Amount,AccOpenDate,LoanLengthMonths,LoanStatus,Category
0,I-00006307127350009978,CL3246299535967411-4949000,14173.66,22839.0,60.0,Current,Car Loan
1,B-00000106932416092604,CL2218500655144454-1006400,16113.63,22444.0,60.0,Late,Car Loan
2,E-00003731750759761782,CL6304442163091154-4247200,36719.85,22425.0,84.0,Current,Car Loan
3,C-00007014713843818754,CL0940160113386811-4367900,14986.34,20683.0,84.0,Current,Car Loan
4,D-00007665674362797290,CL1829489595256747-4344400,10633.20,21438.0,60.0,Current,Car Loan


(solution 2, advanced) Preparing the data by chaining the [eval](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.eval.html#swat.cas.table.CASTable.eval), [query](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.query.html#swat.cas.table.CASTable.query) and [iloc](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.loc.html#swat.cas.table.CASTable.loc) methods on a **CASTable** object.

In [134]:
## Chain the methods on the returned object. The inplace = False parameter returns a CASTable object.
tblPrep = (conn
           .CASTable('loans_raw', caslib = 'casuser')
           .query('Category = "Car Loan" and LoanStatus ne "Fully Paid"', inplace = False)
           .eval('AccOpenDate = mdy(Month, Day, Year)', inplace = False)
           .eval('LoanLengthMonths = LoanLength * 12', inplace = False)
           .loc[:, ['ID', 'AccNumber', 'Amount', 'AccOpenDate', 'LoanLengthMonths','LoanStatus', 'Category']]
)

tblPrep.head()

,ID,AccNumber,Amount,AccOpenDate,LoanLengthMonths,LoanStatus,Category
0,I-00006307127350009978,CL3246299535967411-4949000,14173.66,22839.0,60.0,Current,Car Loan
1,B-00000106932416092604,CL2218500655144454-1006400,16113.63,22444.0,60.0,Late,Car Loan
2,E-00003731750759761782,CL6304442163091154-4247200,36719.85,22425.0,84.0,Current,Car Loan
3,C-00007014713843818754,CL0940160113386811-4367900,14986.34,20683.0,84.0,Current,Car Loan
4,D-00007665674362797290,CL1829489595256747-4344400,10633.20,21438.0,60.0,Current,Car Loan


5. Use the copyTable action to create a new CAS table. Name the new table **CARLOANS** and place it in the **Casuser** caslib.

In [28]:
outTbl = dict(name = 'carloans',
              caslib = 'casuser', 
              replace = True)

tbl.copyTable(casOut = outTbl)

[caslib]

 'CASUSER(student)'

[tableName]

 'CARLOANS'

[casTable]

 CASTable('CARLOANS', caslib='CASUSER(student)')

+ Elapsed: 1.89s, user: 19.6s, sys: 0.884s, mem: 322mb

6. View available in-memory tables in the **Casuser** caslib using the tableInfo action. Confirm the new **CARLOANS** table was created. How many rows are in the new table? How many columns?

In [29]:
## Answer: 838132 rows and 7 columns
conn.tableInfo(caslib = 'casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,LOANS_RAW,16311047,20,0,utf-8,2022-03-21T12:37:46+00:00,2022-03-21T12:37:46+00:00,2022-03-21T12:38:04+00:00,UTF8,1.963485e+09,1.963485e+09,1.963485e+09,0,0,0,0,loans_raw.sashdat,PIVY,0,student,,2021-12-17T18:40:14+00:00,1.955386e+09
1,CARLOANS,838132,7,0,utf-8,2022-03-21T12:38:05+00:00,2022-03-21T12:38:05+00:00,2022-03-21T12:38:05+00:00,UTF8,1.963485e+09,1.963485e+09,1.963485e+09,0,0,0,0,,,0,student,,,NaN


7. Reference the **CARLOANS** CAS table in the variable **carLoans**. Confirm it was referenced successfully.

In [30]:
carLoans = conn.CASTable('carloans', caslib = 'casuser')
carLoans.head(10)

,ID,AccNumber,Amount,AccOpenDate,LoanLengthMonths,LoanStatus,Category
0,I-00006499447310343385,CL0212861374020578-4169400,12632.25,20480.0,60.0,Current,Car Loan
1,J-00009271738699171692,CL4826223445124925-2152200,39791.61,22497.0,60.0,Current,Car Loan
2,C-00002583966117817909,CL3680665963329376-2971100,37379.49,21915.0,60.0,Current,Car Loan
3,J-00001315397601574660,CL5373988857027144-4766500,17725.05,22147.0,60.0,Current,Car Loan
4,G-00006053972726222128,CL5690947889816017-0705600,37680.89,22032.0,60.0,Current,Car Loan
5,F-00007777347383089364,CL8870031002443285-2032800,21566.90,21275.0,60.0,Current,Car Loan
6,A-00002572255444247276,CL8285524963866920-1610700,28837.41,21841.0,60.0,Current,Car Loan
7,J-00004337034830823542,CL9199908324517310-3563200,27021.27,22832.0,36.0,Current,Car Loan
8,F-00005146582631859928,CL5009276715572924-9217200,21163.44,21843.0,84.0,Current,Car Loan
9,J-00009423941555432976,CL0248653960879893-9006400,46784.38,21665.0,84.0,Current,Car Loan


8. How many distint values are in the **Category** and **LoanStatus** columns of the **CARLOANS** CAS table?

In [31]:
## Answers: 
## - 1 distinct value in the Category column (Car Loan)
## - 5 distinct values in the LoanStatus column (Current, Late, Default, Charged Of, Cancelled)

## Using the value_counts method
df = (carLoans
      .Category
      .value_counts())
display(df)

df = (carLoans
      .LoanStatus
      .value_counts())
display(df)

## Using the freq CAS Action
cr = carLoans.freq(inputs = ['Category', 'LoanStatus'])
display(cr)

Car Loan    838132
dtype: int64

Current       730887
Late           45166
Default        30695
Charged Of     19771
Cancelled      11613
dtype: int64

,Column,CharVar,FmtVar,Level,Frequency
0,Category,Car Loan,Car Loan,1,838132.0
1,LoanStatus,Cancelled,Cancelled,1,11613.0
2,LoanStatus,Charged Of,Charged Of,2,19771.0
3,LoanStatus,Current,Current,3,730887.0
4,LoanStatus,Default,Default,4,30695.0
5,LoanStatus,Late,Late,5,45166.0


9. What is the mean of **LoanLengthMonths** of the **CARLOANS** CAS table to the nearest whole integer?

In [32]:
## Answer: LoanLengthMonths mean - 68
(carLoans
 .LoanLengthMonths
 .mean()
 .round())

68.0

10. Terminate the connection to the CAS server if you are not completing the challenge below.

In [95]:
conn.terminate()

### Challenge (optional)
Use the [alterTable](https://go.documentation.sas.com/doc/en/pgmsascdc/v_018/caspg/cas-table-altertable.htm) action to add the [SAS format](https://go.documentation.sas.com/doc/en/pgmsascdc/v_018/leforinforref/n0p2fmevfgj470n17h4k9f27qjag.htm?homeOnFail) MMDDYY10 to the **AccOpenDate** column. Preview the CAS table with the added format.

In [33]:
## Reference the CAS table
carLoans = conn.CASTable('carloans', caslib = 'casuser')

## Preview the table prior to adding the SAS format
df = carLoans.head()
ci = carLoans.columnInfo()
display(df, ci)

## Add a format to the AccOpenDate column
conn.altertable(carLoans, 
                columns = [
                    {'name':'AccOpenDate', 'format':'mmddyy10.'}
                ])

## Preview the table after the SAS format was added
df = carLoans.head()
ci = carLoans.columnInfo()
display(df, ci)

conn.terminate()

,ID,AccNumber,Amount,AccOpenDate,LoanLengthMonths,LoanStatus,Category
0,I-00006499447310343385,CL0212861374020578-4169400,12632.25,20480.0,60.0,Current,Car Loan
1,J-00009271738699171692,CL4826223445124925-2152200,39791.61,22497.0,60.0,Current,Car Loan
2,C-00002583966117817909,CL3680665963329376-2971100,37379.49,21915.0,60.0,Current,Car Loan
3,J-00001315397601574660,CL5373988857027144-4766500,17725.05,22147.0,60.0,Current,Car Loan
4,G-00006053972726222128,CL5690947889816017-0705600,37680.89,22032.0,60.0,Current,Car Loan


,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,ID,,1,varchar,30,30,,0,0
1,AccNumber,,2,varchar,30,30,,0,0
2,Amount,,3,double,8,12,,0,0
3,AccOpenDate,,4,double,8,12,,0,0
4,LoanLengthMonths,,5,double,8,12,,0,0
5,LoanStatus,,6,varchar,10,10,,0,0
6,Category,,7,varchar,16,16,,0,0


,ID,AccNumber,Amount,AccOpenDate,LoanLengthMonths,LoanStatus,Category
0,I-00006499447310343385,CL0212861374020578-4169400,12632.25,2016-01-27,60.0,Current,Car Loan
1,J-00009271738699171692,CL4826223445124925-2152200,39791.61,2021-08-05,60.0,Current,Car Loan
2,C-00002583966117817909,CL3680665963329376-2971100,37379.49,2020-01-01,60.0,Current,Car Loan
3,J-00001315397601574660,CL5373988857027144-4766500,17725.05,2020-08-20,60.0,Current,Car Loan
4,G-00006053972726222128,CL5690947889816017-0705600,37680.89,2020-04-27,60.0,Current,Car Loan


,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,ID,,1,varchar,30,30,,0,0
1,AccNumber,,2,varchar,30,30,,0,0
2,Amount,,3,double,8,12,,0,0
3,AccOpenDate,,4,double,8,12,MMDDYY,10,0
4,LoanLengthMonths,,5,double,8,12,,0,0
5,LoanStatus,,6,varchar,10,10,,0,0
6,Category,,7,varchar,16,16,,0,0
